In [4]:
import re
import itertools
import numpy as np
import pandas as pd
import plotly.express as px
import plotly as py
import plotly.graph_objs as go
from copy import copy, deepcopy
import warnings
# warnings.filterwarnings("ignore")

In [105]:
df = pd.read_csv(r"C:\Users\Dmitrii\Desktop\cow\data\counters.csv")
ans = pd.read_csv(r"C:\Users\Dmitrii\Desktop\cow\data\ancestors.csv")
ansd = {'id':ans['Номер животного'],"father_id":ans['Код Отца']}
ans = pd.DataFrame(data=ansd)


In [17]:
ILLS = ['MAST', 'LAME', 'KETOS', 'METR', 'PARES', 'ILLMISC']
MONTH = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
EVENTS = ["ABORT","TOSCM","NULSCM","DNB","BRED","FRESH","PREG","PREGBEF",
"DRY","DRY2","OPEN","WEIGHT","DEAD","MOVE","SOLD","VAC","VACVIR","FOOTRIM","HEALTH",
"BROKE","POT","ILLMISC","LAME","KETOS","MAST","METR","PARES","RP"]

# График количества здоровых и больных коров

In [110]:
gdf = df[['id', 'event']].groupby('id').agg(list)
gdf['illed'] = gdf['event'].apply(lambda events: any([ill in events for ill in ILLS]))

gdi = gdf[gdf['illed'] == 1]
gdni = gdf[gdf['illed'] == 0]

labels=['Болевшие','Не болевшие']

values=[gdi.count()[0],gdni.count()[0]]
trace=[go.Pie(labels=labels,values=values)]
layout=go.Layout(
    title='Болевшие и не болевшие'
)
fig=go.Figure(data=trace,layout=layout)
fig.show()



# График коров больных маститом на всех больных 

In [111]:
gdf = df[['id', 'event']].groupby('id').agg(list)
gdf['illed'] = gdf['event'].apply(lambda events: any([ill in events for ill in ILLS]))
gdf['mast'] = gdf['event'].apply(lambda events: 'MAST' in events)
gdim = gdf[gdf['mast'] == 1]
gdio = gdf[gdf['illed'] == 1 ]
gdio = gdio[gdio['mast'] == 0] 

labels=['Болела маститом','Болела другой болезнью']

values=[gdim.count()[0],gdio.count()[0]]
trace=[go.Pie(labels=labels,values=values)]
layout=go.Layout(
    title='Доля мастита среди болезней'
)
fig=go.Figure(data=trace,layout=layout)
fig.show()


# График процентного соотношения болезней

In [112]:
gdf = df[['id','event']]
mast = gdf[gdf['event']=='MAST']
mast_c = mast.count()[0]
lame = gdf[gdf['event']=='LAME']
lame_c = lame.count()[0]
ketos = gdf[gdf['event']=='KETOS']
ketos_c = ketos.count()[0]
metr = gdf[gdf['event']=='METR']
metr_c = metr.count()[0]
pares = gdf[gdf['event']=='PARES']
pares_c = pares.count()[0]
illmisc = gdf[gdf['event']=='ILLMISC']
illmisc_c = illmisc.count()[0]

labels=['Мастит','Хромота','Кетоз','Метрит','Парес','Другое']

values=[mast_c,lame_c,ketos_c,metr_c,pares_c,illmisc_c]

trace=[go.Pie(labels=labels,values=values)]
layout=go.Layout(
    title='Статистика болезней'
)
fig=go.Figure(data=trace,layout=layout)
fig.show()

# График болезней по месяцам

In [118]:

gdf = df[['id','event'] + MONTH]

months_c = []
for m in MONTH:
    months_c.append([gdf[(gdf['event'] == ill) & (gdf[m] == 1)].shape[0] for ill in ILLS])    
col1 = []
col2 = []
col3 = []
for i in range(len(MONTH)):
    m = MONTH[i]
    for j in range(len(ILLS)):
        ill = ILLS[j]
        col1.append(m)
        col2.append(ill)
        col3.append(months_c[i][j])
        
long_df = pd.DataFrame({'Месяц' : col1, 'Болезни' : col2, 'Количество' : col3})

fig = px.bar(long_df, x="Месяц", y="Количество", color="Болезни", title="Статистика по месяцам")
fig.show()

# 

# Статистика по ивентам

In [119]:


labels= EVENTS

values=[]
for i in range(len(EVENTS)):
    values.append( df[df['event']==EVENTS[i]].shape[0])
trace=[go.Pie(labels=labels,values=values)]
layout=go.Layout(
    title='Статистика по событиям'
)
fig=go.Figure(data=trace,layout=layout)
fig.show()

In [36]:
ans.head()

,id,father_id
0,1,-
1,2,-
2,3,-
3,4,-
4,6,-


# График по родителям

In [109]:
gdf = df[['id', 'event']]
gdf['illed'] = gdf['event'].apply(lambda events: any([ill in events for ill in ILLS]))
gdf['mast']  = gdf['event'].apply(lambda events: 'MAST' in events)
gdim = gdf[gdf['mast'] == 1]
illed_id = gdim['id'].unique().tolist()


ans['ill'] = ans['id'].apply(lambda x: x in illed_id)

# ans = ans[ans['ill']==1]
# ans.head()

#ans.head()
# illed = df[['id', 'event']].groupby('id').agg(list)
# nes_ids=[]

# ans['illkids']

# par.head()
# par=ans[ans["id"]==illed['id']]
new = ans
new = new.groupby('father_id').sum().reset_index()
new.head()
fig = px.bar(new, x='father_id', y='ill')
fig.show()

C:\Users\Dmitrii\AppData\Local\Temp\ipykernel_15568\1914791118.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Dmitrii\AppData\Local\Temp\ipykernel_15568\1914791118.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Хромота масит все ост


In [122]:
gdf = df[['id', 'event']].groupby('id').agg(list)
gdf['illed'] = gdf['event'].apply(lambda events: any([ill in events for ill in ILLS]))
gdf['mast'] = gdf['event'].apply(lambda events: 'MAST' in events)
gdf['lame'] = gdf['event'].apply(lambda events: 'LAME' in events)
gdim = gdf[gdf['mast'] == 1]
gdil = gdf[gdf['lame'] == 1]
gdio = gdf[gdf['illed'] == 1 ]
gdio = gdio[gdio['mast'] == 0] 
gdio = gdio[gdio['lame'] == 0]

labels=['Болела маститом','Болела хромотой','Болела другой болезнью']

values=[gdim.count()[0],gdil.count()[0],gdio.count()[0]]
trace=[go.Pie(labels=labels,values=values)]
layout=go.Layout(
    title='Доля мастита и хромоты среди болезней'
)
fig=go.Figure(data=trace,layout=layout)
fig.show()

# Ans

In [187]:
an=pd.read_csv(r"C:\Users\Dmitrii\Desktop\cow\data\ancestors_stat.csv")
anfull = pd.read_csv(r"C:\Users\Dmitrii\Downloads\Telegram Desktop\ancestors_full_stat.csv")





In [191]:
fathers = an.columns.to_list()
allnum=[]
del fathers[0]

num=[]
for father in fathers:
    num.append( an[father][0])

for father in fathers:
    allnum.append( anfull[father][0])



num = np.array(num)
print(num.mean(), ' ')
print(num.std())

datapers = []
for i in range (len(allnum)):
    datapers .append(num[i]/allnum[i])

datadf=pd.DataFrame({'Имя':fathers,'Процент':datapers})

fig = px.bar(datadf, x='Имя', y='Процент')
fig.show()


['1H11344', '777H10574', '777H10778', '1H11673', '1H11917', '1H11978', '777H10690', '777H10771', 'PLUTON', 'CALGARY', '1H13162', '777H10777', '1H12902', '777H11079', '1H12490', '777H10915', '1H11372', '777H11243', '1H13471', '1H11913', '1H13411', '1H15057', '1H13456', '1H14138', '501H15000', '501H15074', 'ANSVER', 'FEIM', '1H13878', '501H14018', '1H15185', '1H15184', '1H15132', '501H15132', 'LOCKLYN', 'OSOFINE', 'ESQUIRE', 'FIGHTER', 'NETWORT', 'TOMAH', 'DOLMAN', 'GALAXY', 'DONNIE', 'TRINITY', 'BULL', 'ERIC', 'EDEN', 'PILOT', 'ICEMAN', 'KAMPMAN', 'CHARMAN', 'NATWORT', 'RELANT', 'GRAYTON', 'LADDY', 'BONJOVI', 'EDENS', 'YUENGLIN', 'DISTANCE', 'PADLOCK', 'TOUCH', 'ROSS', '4MOCT', 'NETWORTH', 'BOJAK', 'KLEIK', 'REMOTE', 'LAMANSCH', 'LAURIN', 'MODEL', 'DELKAMPO', 'ALMOND', 'EMERALD', 'CLAYK', 'GILLETTE', 'CHAPEL', 'COUNTRY', 'WYN', 'TAK', 'BUILDER', 'LOTTIE', 'OSOFINES', 'LARIATS', 'BALANCE', 'BRIAR', 'BALANS', 'TREIDER', 'DEKART', 'ALFA', 'TYSON', 'SCHKODA', 'AMSTRNGS', 'SHATL', 'ALABAMA',

C:\Users\Dmitrii\AppData\Local\Temp\ipykernel_15568\2789903973.py:20: RuntimeWarning:

invalid value encountered in longlong_scalars

